In [2]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import learning_curve
import keras
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf

Using TensorFlow backend.


# Data Processing

In [3]:
filepath = '../../Datasets/SICK/SICK.txt'
data = pd.read_csv(filepath, sep='\t')

In [4]:
data.head()

,pair_ID,sentence_A,sentence_B,entailment_label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset,SemEval_set
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,NEUTRAL,4.5,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR,TRAIN
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,NEUTRAL,3.2,A_contradicts_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR,TRAIN
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,ENTAILMENT,4.7,A_entails_B,B_entails_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRAIN
3,4,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,CONTRADICTION,3.6,A_contradicts_B,B_contradicts_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRIAL
4,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,NEUTRAL,3.4,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRAIN


In [5]:
num_sentences = data.shape[0]
num_labels = 3

We will extract the Premise and Hypothesis along with the Entailment Label to construct our dataset.

In [6]:
X = data[['sentence_A','sentence_B']]
Y = data['entailment_label']

We can process our label encoding now, but our sentences are not in a format which can be encoded.

In [7]:
le = preprocessing.LabelEncoder().fit(Y)
Y = np_utils.to_categorical(le.transform(Y))
print(Y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


sentence_A holds the Premise while sentence_B holds the Hypothesis data.

To tokenize our corpus we will need to stack these two columns into a single series.

In [8]:
stacked = pd.concat([X['sentence_A'], X['sentence_B']], axis=0)

In [9]:
# Convert question corpus into sequential encoding for LSTM
vocab_size = 512
sequence_length = 16
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(stacked)
sequences = tokenizer.texts_to_sequences(stacked)
x_text = sequence.pad_sequences(sequences, maxlen=sequence_length)

premise_sequences = x_text[:data.shape[0]]
hypothesis_sequences = x_text[data.shape[0]:]

# Neural Network

In [10]:
epochs = 100
batch_size = 64
learning_rate = .001
dense_units = 128
dropout_rate = .5

In [11]:
embedding_layer = Embedding(vocab_size, 128)
premise_inputs = Input(shape=(sequence_length,))
premise_embedding = embedding_layer(premise_inputs)
premise_outputs, state_h, state_c = LSTM(128, return_state=True)(premise_embedding)

hypothesis_inputs = Input(shape=(sequence_length,))
hypothesis_embedding = embedding_layer(hypothesis_inputs)
hypothesis_outputs = LSTM(128)(hypothesis_embedding, initial_state=[state_h, state_c])

dense = Dense(dense_units, activation='relu', kernel_initializer=keras.initializers.he_normal(seed=None))(hypothesis_outputs)
bn = BatchNormalization()(dense)
dropout = Dropout(dropout_rate)(bn)
outputs = Dense(num_labels, activation='softmax')(dropout)

In [12]:
model = Model(inputs=[premise_inputs, hypothesis_inputs], outputs=outputs)
print(model.summary())

model.compile(optimizer=Adam(lr=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([premise_sequences, hypothesis_sequences], Y, batch_size=batch_size, epochs=epochs, shuffle=True, validation_split = 0.2)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 16)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 16)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 16, 128)      65536       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 131584      embedding_1[0][0]          

/Users/tzhang270/Desktop/Projects/ML/Models/Attention-LSTM/venv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7872 samples, validate on 1968 samples
Epoch 1/100
7872/7872 [==============================] - 7s 951us/step - loss: 0.9303 - accuracy: 0.5201 - val_loss: 0.7986 - val_accuracy: 0.6753
Epoch 2/100
7872/7872 [==============================] - 6s 721us/step - loss: 0.7747 - accuracy: 0.6071 - val_loss: 0.7665 - val_accuracy: 0.6753
Epoch 3/100
7872/7872 [==============================] - 6s 706us/step - loss: 0.7389 - accuracy: 0.6184 - val_loss: 0.7721 - val_accuracy: 0.6753
Epoch 4/100
7872/7872 [==============================] - 6s 726us/step - loss: 0.7147 - accuracy: 0.6353 - val_loss: 0.7267 - val_accuracy: 0.6728
Epoch 5/100
7872/7872 [==============================] - 6s 780us/step - loss: 0.6885 - accuracy: 0.6533 - val_loss: 0.7408 - val_accuracy: 0.6453
Epoch 6/100
7872/7872 [==============================] - 7s 862us/step - loss: 0.6815 - accuracy: 0.6535 - val_loss: 0.7760 - val_accuracy: 0.6103
Epoch 7/100
7872/7872 [==============================] - 6s 824us/step

7872/7872 [==============================] - 6s 708us/step - loss: 0.1089 - accuracy: 0.9605 - val_loss: 2.1713 - val_accuracy: 0.6169
Epoch 57/100
7872/7872 [==============================] - 6s 712us/step - loss: 0.1094 - accuracy: 0.9609 - val_loss: 2.3805 - val_accuracy: 0.6067
Epoch 58/100
7872/7872 [==============================] - 6s 718us/step - loss: 0.1227 - accuracy: 0.9564 - val_loss: 2.0858 - val_accuracy: 0.6194
Epoch 59/100
7872/7872 [==============================] - 6s 727us/step - loss: 0.1159 - accuracy: 0.9562 - val_loss: 2.2386 - val_accuracy: 0.6062
Epoch 60/100
7872/7872 [==============================] - 6s 699us/step - loss: 0.1046 - accuracy: 0.9620 - val_loss: 2.3378 - val_accuracy: 0.5935
Epoch 61/100
7872/7872 [==============================] - 6s 699us/step - loss: 0.1168 - accuracy: 0.9546 - val_loss: 1.9805 - val_accuracy: 0.6321
Epoch 62/100
7872/7872 [==============================] - 6s 700us/step - loss: 0.1085 - accuracy: 0.9620 - val_loss: 2.1770 